In [1]:
import pandas as pd
import numpy as np
import pydicom 
import os
import matplotlib.pyplot as plt

import tensorflow as tf
#import keras
from keras.models import Model, Sequential
from keras.layers import Concatenate, Dense, Input, concatenate, BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Conv3D, MaxPool3D, AveragePooling3D

from sklearn.preprocessing import MinMaxScaler

In [2]:
def load_best_slices(path):
    ind = []
    for file in os.listdir(path):
        if file.split('.')[1] == 'npy':
            ind.append(file.split('.')[0])
        
    df = pd.DataFrame(index = ind, columns= ['CT'])
    for ind in df.index:
        df.loc[ind].CT = np.load(path + ind + '.npy')
    
    return df

df = load_best_slices('lung_chunks/')

In [3]:
def custom_loss_function():

    def loss_function(y_true, y_pred):

        diff = abs(y_pred-y_true)
        
        return tf.reduce_mean(diff**4, axis=-1)**.25
    
    return loss_function

def custom_metric_function():

    def metric_function(y_true, y_pred):
        
        diff = abs(y_pred-y_true)
        diff = tf.where(diff > 7.5, 7.5, diff)
        diff = tf.where(diff < 0.5, 0.5, diff)
        
        return tf.reduce_mean(diff, axis=-1)
    
    return metric_function

In [4]:
df = df.reset_index()
df = df.rename(columns = {'index' : 'Patient'})

In [5]:
features = pd.read_csv('features.csv')
df = df.merge(features, on= 'Patient')

In [6]:
linear_data_all = pd.read_csv('patient_slope_intercept.csv', index_col=0)
result = pd.DataFrame(index = df.Patient, columns = ['slope'])
    
for ind in result.index:
    result.loc[ind].slope = linear_data_all.loc[ind].slope

df = df.merge(result, on='Patient')
df.iloc[:,2:] = df.iloc[:,2:] #.astype('float32')

In [7]:
#df.head()

In [8]:
dataset = df.values[:,1:]

In [9]:
scaler_features = MinMaxScaler()
dataset[:,1:-1] = scaler_features.fit_transform(dataset[:,1:-1])

In [10]:
y = dataset[:,-1].copy().astype('float32')

In [11]:
y

array([-1.4362868e+01, -1.3071039e+00, -7.3706927e+00, -8.5644579e+00,
        1.3588895e+00, -4.2086034e+00,  1.4789916e+00, -3.7748799e-02,
       -1.4086339e+00, -2.8182575e+01,  4.1800714e+00,  9.0522079e+00,
        2.5571496e+00, -1.4488882e+00, -1.5641643e+00, -2.5840535e+00,
       -3.0797420e+00, -1.8381779e+01, -1.6121746e+01, -8.9318962e+00,
       -6.5893281e-01, -3.3735905e+00, -5.9531841e+00,  6.9377160e-01,
       -2.3159204e+00, -8.9127941e+00, -1.9162534e-01, -5.1311207e+00,
       -1.7042803e+01, -5.6847239e-01, -5.7333183e+00, -2.3287500e+01,
        7.3312583e+00, -5.8735952e+00, -8.1894598e+00, -1.0402375e+00,
       -2.0166771e+00, -9.3818474e+00, -1.0939918e+01, -1.0805558e+00,
       -3.1304660e+00, -9.5680344e-01, -3.4443293e+00, -9.2148857e+00,
       -8.0079710e-01, -4.3741259e-01, -4.6877728e+00, -1.5106828e+00,
       -3.4103143e+00, -2.3561151e+00, -4.7401042e+00, -8.5430784e+00,
       -1.5101547e+00,  1.8965999e+00, -9.2992169e-01, -7.5789595e+00,
      

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset[:,:-1], y, test_size=0.2)

In [13]:
# lungs_train = []
# for i in range(X_train[:,0].shape[0]):
#     lungs_train.append(X_train[:,0][0])
# lungs_train = tf.convert_to_tensor(lungs_train)
# lungs_train = tf.reshape(lungs_train, (X_train[:,0].shape[0], 7, 512, 512, 1))

In [14]:
lungs_train = []
for i in range(X_train[:,0].shape[0]):
    lungs_train.append(X_train[:,0][0]) #.astype(np.float32))

lungs_train = np.array(lungs_train) #.astype(np.float32)   
lungs_train = lungs_train.reshape(X_train[:,0].shape[0], 7, 512, 512, 1)

lungs_test = []
for i in range(X_test[:,0].shape[0]):
    lungs_test.append(X_test[:,0][0]) #.astype(np.float32))

lungs_test = np.array(lungs_test) #.astype(np.float32)    
lungs_test = lungs_test.reshape(X_test[:,0].shape[0], 7, 512, 512, 1)

In [15]:
#features_train = []
#for i in range(X_train[:,1:].shape[0]):
#    features_train.append(X_train[i,1:]) #.astype(np.float32))
#    
#features_test = []
#for i in range(X_test[:,1:].shape[0]):
#    features_test.append(X_test[i,1:]) #.astype(np.float32))
    
features_train = []
for i in range(X_train[:,1:].shape[0]):
    features_train.append(X_train[:,1:][0])
features_train = tf.convert_to_tensor(features_train)
features_train = tf.reshape(features_train, (features_train.shape[0],8))

features_test = []
for i in range(X_test[:,1:].shape[0]):
    features_test.append(X_test[:,1:][0])
features_test = tf.convert_to_tensor(features_test)
features_test = tf.reshape(features_test, (features_test.shape[0],8))

In [16]:
features_train = []
for i in range(X_train[:,1:].shape[0]):
    features_train.append(X_train[:,1:][0])
features_train = np.array(np.array(features_train)).astype(np.float32)
features_train = features_train.reshape(features_train.shape[0], 8)

features_test = []
for i in range(X_test[:,1:].shape[0]):
    features_test.append(X_test[:,1:][0])
features_test = np.array(np.array(features_test)).astype(np.float32)
features_test = features_test.reshape(features_test.shape[0], 8)

In [49]:
inp_conv = Input(shape = (7,512,512,1), name='lungs')

model_conv = Conv3D(16, kernel_size=(1,2,2), strides=(1,1,1), padding='valid', activation='relu')(inp_conv)
model_conv = Conv3D(8, kernel_size=(1,3,3), strides=(1,1,1), padding='valid', activation='relu')(model_conv)
model_conv = BatchNormalization()(model_conv)
model_conv = MaxPool3D(pool_size=(2,3,3))(model_conv)
model_conv = Dropout(0.25)(model_conv)
model_conv = Conv3D(16, kernel_size=(2,1,2), strides=(1,1,1), padding='valid', activation='relu')(model_conv)
model_conv = Conv3D(16, kernel_size=(2,2,1), strides=(1,1,1), padding='valid', activation='relu')(model_conv)
model_conv = BatchNormalization()(model_conv)
model_conv = AveragePooling3D(pool_size=(1,3,3))(model_conv)
model_conv = Dense(8, activation='relu')(model_conv)
model_conv = MaxPool3D(pool_size=(1,3,3))(model_conv)
model_conv = Dropout(0.25)(model_conv)
model_conv = Flatten()(model_conv)
model_conv = Dense(8, activation='relu')(model_conv)
outp_conv = Dense(8, activation='sigmoid')(model_conv)

inp_feat = Input(shape = (8,), name='features')
model_feat = Dense(16, activation='relu')(inp_feat)
model_feat = Dense(32, activation='relu')(model_feat)
outp_feat = Dense(8, activation='sigmoid')(model_feat)

model_conc = concatenate([outp_conv, outp_feat])
model_conc = Dense(16, activation='relu')(model_conc)
model_conc = Dense(32, activation='relu')(model_conc)
model_conc = Dense(8, activation='relu')(model_conc)
output = Dense(1, activation='linear')(model_conc)

model = Model(inputs=[inp_conv, inp_feat], outputs=output, name="cnn_nn_model")

In [50]:
model.summary()

Model: "cnn_nn_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lungs (InputLayer)              [(None, 7, 512, 512, 0                                            
__________________________________________________________________________________________________
conv3d_44 (Conv3D)              (None, 7, 511, 511,  80          lungs[0][0]                      
__________________________________________________________________________________________________
conv3d_45 (Conv3D)              (None, 7, 509, 509,  1160        conv3d_44[0][0]                  
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 7, 509, 509,  32          conv3d_45[0][0]                  
_______________________________________________________________________________________

In [51]:
model.compile(loss=custom_loss_function(), metrics=[custom_metric_function()], optimizer='adam')
#model.compile(loss='mse', metrics=['mse'],optimizer='adam')

In [52]:
model.fit(
    {"lungs": lungs_train, "features": features_train},
    y_train,
    epochs=200, #experimenta mudar este número
    batch_size=8,
    validation_data = ({"lungs": lungs_test, "features": features_test}, y_test)
)

Epoch 1/200
18/18 [==============================] - 14s 788ms/step - loss: 5.2682 - metric_function: 4.1313 - val_loss: 6.0476 - val_metric_function: 4.5268
Epoch 2/200
18/18 [==============================] - 12s 661ms/step - loss: 4.9775 - metric_function: 3.9482 - val_loss: 5.7293 - val_metric_function: 4.3228
Epoch 3/200
18/18 [==============================] - 12s 664ms/step - loss: 4.7044 - metric_function: 3.7755 - val_loss: 5.4118 - val_metric_function: 4.0984
Epoch 4/200
18/18 [==============================] - 12s 661ms/step - loss: 4.4438 - metric_function: 3.6084 - val_loss: 5.1687 - val_metric_function: 3.9282
Epoch 5/200
18/18 [==============================] - 12s 660ms/step - loss: 4.2757 - metric_function: 3.5059 - val_loss: 4.9912 - val_metric_function: 3.8285
Epoch 6/200
18/18 [==============================] - 12s 662ms/step - loss: 4.1938 - metric_function: 3.4842 - val_loss: 4.9316 - val_metric_function: 3.7945
Epoch 7/200
18/18 [==============================] -

Epoch 51/200
18/18 [==============================] - 12s 665ms/step - loss: 4.1902 - metric_function: 3.5021 - val_loss: 4.9163 - val_metric_function: 3.7700
Epoch 52/200
18/18 [==============================] - 12s 664ms/step - loss: 4.1896 - metric_function: 3.5003 - val_loss: 4.9232 - val_metric_function: 3.7839
Epoch 53/200
18/18 [==============================] - 12s 666ms/step - loss: 4.1984 - metric_function: 3.5014 - val_loss: 4.9249 - val_metric_function: 3.7873
Epoch 54/200
18/18 [==============================] - 12s 670ms/step - loss: 4.1914 - metric_function: 3.5077 - val_loss: 4.9138 - val_metric_function: 3.7631
Epoch 55/200
18/18 [==============================] - 12s 667ms/step - loss: 4.1928 - metric_function: 3.5092 - val_loss: 4.9185 - val_metric_function: 3.7745
Epoch 56/200
18/18 [==============================] - 12s 673ms/step - loss: 4.1879 - metric_function: 3.5008 - val_loss: 4.9172 - val_metric_function: 3.7718
Epoch 57/200
18/18 [==========================

18/18 [==============================] - 12s 666ms/step - loss: 4.1905 - metric_function: 3.5026 - val_loss: 4.9178 - val_metric_function: 3.7731
Epoch 103/200
18/18 [==============================] - 12s 665ms/step - loss: 4.1920 - metric_function: 3.5117 - val_loss: 4.9138 - val_metric_function: 3.7624
Epoch 104/200
18/18 [==============================] - 12s 663ms/step - loss: 4.1888 - metric_function: 3.5037 - val_loss: 4.9215 - val_metric_function: 3.7805
Epoch 105/200
18/18 [==============================] - 12s 664ms/step - loss: 4.2032 - metric_function: 3.5030 - val_loss: 4.9258 - val_metric_function: 3.7888
Epoch 106/200
18/18 [==============================] - 12s 663ms/step - loss: 4.1962 - metric_function: 3.5020 - val_loss: 4.9225 - val_metric_function: 3.7824
Epoch 107/200
18/18 [==============================] - 12s 664ms/step - loss: 4.1868 - metric_function: 3.5000 - val_loss: 4.9155 - val_metric_function: 3.7684
Epoch 108/200
18/18 [==============================] -

18/18 [==============================] - 12s 674ms/step - loss: 4.1918 - metric_function: 3.5154 - val_loss: 4.9138 - val_metric_function: 3.7609
Epoch 154/200
18/18 [==============================] - 12s 672ms/step - loss: 4.1968 - metric_function: 3.5196 - val_loss: 4.9152 - val_metric_function: 3.7679
Epoch 155/200
18/18 [==============================] - 12s 686ms/step - loss: 4.1912 - metric_function: 3.4995 - val_loss: 4.9207 - val_metric_function: 3.7789
Epoch 156/200
18/18 [==============================] - 12s 672ms/step - loss: 4.1867 - metric_function: 3.4980 - val_loss: 4.9165 - val_metric_function: 3.7704
Epoch 157/200
18/18 [==============================] - 12s 662ms/step - loss: 4.1902 - metric_function: 3.5113 - val_loss: 4.9138 - val_metric_function: 3.7615
Epoch 158/200
18/18 [==============================] - 12s 689ms/step - loss: 4.2072 - metric_function: 3.5083 - val_loss: 4.9221 - val_metric_function: 3.7817
Epoch 159/200
18/18 [==============================] -

In [53]:
model.predict(([lungs_test[0:5,:,:,:,:], features_test[0:5,:]]))

array([[-3.771681],
       [-3.771681],
       [-3.771681],
       [-3.771681],
       [-3.771681]], dtype=float32)

In [54]:
y_test[0:5]

array([-18.381779 ,   9.052208 ,  -4.740104 ,  -3.6651213,  -3.3800275],
      dtype=float32)